In [1]:
%load_ext autoreload
%autoreload 2
import energyderivatives as ed

In [2]:
import matplotlib.pyplot as plt

In [3]:
import numpy as np

In [4]:
S = 100
K = 100
t = 1 / 12
sigma = 0.4
r = 0.10
b = 0.1

dt = 1 / 360

path_length = 30
mc_samples = 5000
mc_loops = 50

In [5]:
inno = ed.monte_carlo_options.NormalSobolInnovations
path = ed.monte_carlo_options.WienerPath
payoff = ed.monte_carlo_options.PlainVanillaPayoff

In [15]:
mc = ed.monte_carlo_options.MonteCarloOption(
    mc_loops,
    path_length,
    mc_samples,
    dt,
    S,
    K,
    t,
    r,
    b,
    sigma,
    inno,
    path,
    payoff,
    # eps=eps,
    trace=False,
    antithetic=True,
    standardization=False
)


In [34]:
mc.call()

/home/bcho/miniconda3/envs/main/lib/python3.9/site-packages/scipy/stats/_qmc.py:1078: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  warnings.warn("The balance properties of Sobol' points require"


array([4.99378655, 4.98084417, 5.01998475, 5.01980064, 5.01667424,
       5.02661883, 5.02971122, 5.00945124, 5.00418056, 5.00711529,
       4.9929845 , 4.99133997, 4.99398536, 4.99460228, 4.99088578,
       5.01279286, 4.99316601, 5.0298758 , 5.00138746, 4.98545674,
       5.03377748, 5.03239799, 5.02813276, 5.03007415, 5.0062853 ,
       4.98227852, 4.99798504, 4.99449782, 5.00864305, 5.0239836 ,
       5.04616129, 5.02069839, 4.99248466, 4.97054339, 5.00301501,
       5.01861537, 5.0269596 , 5.03049148, 5.00484745, 5.0054288 ,
       5.02697772, 5.04870249, 5.02769661, 4.99254376, 5.03397308,
       5.02173478, 5.02104162, 5.04959698, 5.02082948, 5.01562606])

In [35]:
mc.put()

/home/bcho/miniconda3/envs/main/lib/python3.9/site-packages/scipy/stats/_qmc.py:1078: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  warnings.warn("The balance properties of Sobol' points require"


array([4.16143405, 4.15059512, 4.18293501, 4.18882626, 4.167561  ,
       4.17426059, 4.181604  , 4.20553457, 4.17944064, 4.19303112,
       4.16726067, 4.18860452, 4.19335894, 4.19321179, 4.16299317,
       4.17056684, 4.19578788, 4.15307073, 4.20679072, 4.1371889 ,
       4.19080757, 4.19164989, 4.18436994, 4.16885958, 4.18381968,
       4.16787141, 4.21723183, 4.17648198, 4.22373807, 4.1949364 ,
       4.18610996, 4.16425411, 4.16809558, 4.1851566 , 4.17338906,
       4.19812224, 4.18938951, 4.19214795, 4.18544419, 4.16411992,
       4.17462162, 4.19069208, 4.19102653, 4.17666814, 4.19260901,
       4.20374582, 4.1677814 , 4.18368977, 4.19691813, 4.15938752])